In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("final_train_nooutliers.csv")
test = pd.read_csv("final_test2.csv")

In [3]:
print("No of uniques in variant ", train['variant'].nunique())
print("No of uniques in model", train['model'].nunique())

No of uniques in variant  3085
No of uniques in model 355


## 1. Fixing 'model'

### Getting rid of the brand name from model

In [4]:
two_worded_brand = []
one_worded_brand = []

In [5]:
for brand in train['oem'].unique():
    if len(brand.split(" ")) == 2:
        two_worded_brand.append(brand)
    else:
        one_worded_brand.append(brand)

In [6]:
print("Total no. of brands: ", len(train['oem'].unique()))
print("Total no. of double worded brands: ", len(one_worded_brand))
print("Total no. of single worded brands: ", len(two_worded_brand))

Total no. of brands:  31
Total no. of double worded brands:  28
Total no. of single worded brands:  3


In [7]:
def clean_model(row):
    if row['oem'] in two_worded_brand:
        return ' '.join(row['model'].split()[2:])
    elif row['oem'] in one_worded_brand:
        return ' '.join(row['model'].split()[1:])

In [8]:
train['model'] = train.apply(clean_model, axis = 1)
test['model'] = test.apply(clean_model, axis = 1)

In [9]:
if 'land rover' in two_worded_brand:
    print("aloha")

aloha


In [10]:
train['model'].value_counts()

city                 1509
i20                  1215
swift                1200
wagon r              1121
swift dzire          1000
                     ... 
x4                      1
venture                 1
v90 cross country       1
x-trail                 1
f-type                  1
Name: model, Length: 354, dtype: int64

In [11]:
test['model'].value_counts()

city           183
i20            173
swift          153
wagon r        138
swift dzire    110
              ... 
meridian         1
tt               1
baleno rs        1
bolt             1
scorpio n        1
Name: model, Length: 250, dtype: int64

## 2. Fixing 'Variant'

### Getting rid of 'petrol' and 'diesel' from the name

In [12]:
defaulters = []
def find_defaulters(row):
    if(('diesel' in row['variant'].split()) | ('petrol' in row['variant'].split())):
        defaulters.append(row['variant'])

In [13]:
train.apply(find_defaulters, axis = 1)

0        None
1        None
2        None
3        None
4        None
         ... 
31060    None
31061    None
31062    None
31063    None
31064    None
Length: 31065, dtype: object

In [14]:
defaulters = list(set(defaulters))
len(defaulters)

358

In [15]:
defaulters

['v cvt petrol',
 'petrol exi',
 'diesel ls',
 'w6 amt diesel',
 'gls diesel at',
 'titanium diesel',
 '2.0 limited opt diesel',
 'style diesel mt 7 str',
 'diesel highline 1.2l',
 'r-dynamic s petrol',
 '1.5 diesel titanium bsiv',
 'w6 amt diesel sunroof',
 'mx diesel',
 '1.5 gtx plus diesel at',
 'diesel e4',
 'prestige executive 7-seater diesel at',
 'zeta diesel',
 'magna petrol bsiv',
 'signature edition diesel bsiv',
 't petrol',
 'diesel style',
 '1.2l diesel classic',
 'lx 4-str convert top diesel at',
 'trend petrol bsiv',
 'petrol ls 8 seater',
 'd petrol',
 'vx cvt diesel bsiv',
 'luxury plus diesel',
 '1.6 duratec petrol',
 'new diesel highline',
 'zxi plus petrol',
 'petrol rxs',
 'prestige diesel',
 '1.2l diesel elegance',
 'vx petrol bsiv',
 'ax 6-str soft top diesel',
 '1.5 diesel titanium',
 '1.6 ex diesel',
 '1.5 petrol rxl',
 'platinum diesel at',
 's cvt petrol bsiv',
 '1.5 petrol titanium plus bsiv',
 '110ps diesel rxz pack',
 'ambulance petrol ac plus htr',
 'z8l 

In [16]:
def clean_variant(row):
    if (('diesel' in row['variant'].split()) | ('petrol' in row['variant'].split())):
        splitted = row['variant'].split()
        final = [s for s in splitted if s not in ['diesel', 'petrol']]
        return ' '.join(final)
    else:
        return row['variant']

In [17]:
train['variant'] = train.apply(clean_variant, axis = 1)
test['variant'] = test.apply(clean_variant, axis = 1)

In [18]:
defaulters = []
train.apply(find_defaulters, axis = 1)

0        None
1        None
2        None
3        None
4        None
         ... 
31060    None
31061    None
31062    None
31063    None
31064    None
Length: 31065, dtype: object

In [19]:
defaulters

[]

In [21]:
print("Previously: ")
print("No of uniques in variant: 3085")
print("Now: ")
print("No of uniques in variant: ", train['variant'].nunique())

Previously: 
No of uniques in variant: 3085
Now: 
No of uniques in variant:  2911


In [22]:
train.to_csv('data_entry_train.csv', index = False)

## 2. Dealing with categorical columns

In [167]:
#Finding the categorical columns
cat = [col for col in train.columns if train[col].dtype == 'O']
cat

['transmission',
 'fuel',
 'oem',
 'model',
 'variant',
 'Valve Configuration',
 'Drive Type',
 'Fuel Supply System',
 'body',
 'Gear Box',
 'Steering Type',
 'Front Brake Type',
 'Rear Brake Type',
 'Tyre Type',
 'exterior_color']

### 2.1. Transmission

In [168]:
comparison = {}
for col in cat:
    varname = f'{col}_price'
    comparison[varname] = train.groupby(f'{col}')['listed_price'].agg([np.mean, min, max])

In [169]:
comparison['transmission_price']
#Clearly automatic cars are more expensive

,mean,min,max
transmission,,,
automatic,1.469067e+06,60000.0,28300000.0
manual,5.081978e+05,11963.0,3200000.0


In [170]:
def convert_transmission(row):
    if row['transmission'] == 'manual':
        return 0
    if row['transmission'] == 'automatic':
        return 1

### 2.2. Fuel

In [171]:
comparison['fuel_price']
#Also upon further research clear hierarchy is present
#electric > diesel > petrol > cng > lpg

,mean,min,max
fuel,,,
cng,4.254505e+05,50000.0,1200000.0
diesel,9.493799e+05,11963.0,18500000.0
electric,1.498626e+06,236204.0,9000000.0
lpg,1.740199e+05,50000.0,350000.0
petrol,5.726273e+05,25000.0,28300000.0


In [172]:
def convert_fuel(row):
    if row['fuel'] == 'lpg':
        return 0
    if row['fuel'] == 'cng':
        return 1
    if row['fuel'] == 'petrol':
        return 2
    if row['fuel'] == 'diesel':
        return 3
    if row['fuel'] == 'electric':
        return 4

### Drive Type

In [173]:
comparison['Drive Type_price']
#Upon research the hierarchy is same
#awd > rwd > 2wd > fwd

,mean,min,max
Drive Type,,,
2wd,7.967986e+05,40000.0,4600000.0
awd,2.114139e+06,170000.0,28300000.0
fwd,5.807789e+05,11963.0,6585000.0
rwd,1.379340e+06,50000.0,10000000.0


In [174]:
def convert_DriveType(row):
    if row['Drive Type'] == 'awd':
        return 4
    if row['Drive Type'] == 'rwd':
        return 3
    if row['Drive Type'] == '2wd':
        return 2
    if row['Drive Type'] == 'fwd':
        return 1

In [177]:
train['transmission'] = train.apply(convert_transmission, axis = 1)
train['Drive Type'] = train.apply(convert_DriveType, axis = 1)
train['fuel'] = train.apply(convert_fuel, axis = 1)

test['transmission'] = test.apply(convert_transmission, axis = 1)
test['Drive Type'] = test.apply(convert_DriveType, axis = 1)
test['fuel'] = test.apply(convert_fuel, axis = 1)

In [179]:
num_cols = [col for col in train if train[col].dtype != 'O']
num_cols

['myear',
 'transmission',
 'fuel',
 'km',
 'Turbo Charger',
 'Kerb Weight',
 'Drive Type',
 'Seats',
 'Top Speed',
 'Acceleration',
 'Doors',
 'Cargo Volume',
 'owner_type',
 'Max Torque Delivered',
 'listed_price',
 'avg_measure',
 'Features',
 'Valves',
 'Tread']

In [180]:
train.to_csv('train3.csv', index = False)
test.to_csv('test3.csv', index = False)